<a href="https://colab.research.google.com/github/HanifFaishalH/MachineLearning/blob/main/UTS_2341720116_Hanif_Faishal_Hilmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UTS

## Nama     : Hanif Faishal Hilmi

## Kelas     : TI-3F

## NIM     : 2341720116

# Studi Kasus 3 - Heart Diseases

- Studi kasus ini tentang ciri penyakit liver.
- Untuk mahasiswa dengan no absen 3, 6, 9, 12, dst
- Lakukan beberapa tahapan berikut dan lakukan proses analisis terhadap hasil yang dihasilkan.

## Pra Pengolahan Data (10 poin)

1. Load data (0 poin)
2. Cek deskripsi data apakah menggandung nilai yang hilang. Jika ya, sebutkan berapa dan pada fitur yang mana. (5 poin)
3. Drop fitur yang tidak relevan untuk proses clustering. (5 poin)

## Feature Selection (20 poin)

1. Pilih 5 variabel terbaik menurut Anda dari seluruh variabel yang ada. (10 poin)
2. Bagaimana cara Anda memilih 5 variabel tersebut? Kemukakan alasan Anda. (10 poin)

## Clustering - 1 (30 poin)

1. Lakukan pembuatan model cluster dengan menggunakan KMeans dengan jumlah cluster mulai 1 hingga 10. (10 poin)
2. Buat diagram elbow, tentukan jumlah cluster terbaik menurut elbow method dan berikan alasan. (10 poin)
3. Hitung nilai silhouette coefficient dari k=1 hingga k=10. Berdassarkan nilai silhouette, berapa jumlah cluster terbaik? Apakah sama dengan metode elbow? (10 poin)

NB: Anda diperbolehkan untuk melakukan proses normalisasi/standardisasi sebelum pembuatan model clustering. Lakukan pengamatan terhadap hasil clustering.

## Clustering - 2 (20 poin)

1. Lakukan pembuatan model cluster dengan menggunakan DBSCAN. Berapa jumlah cluster yang dihasilkan? (10 poin)
2. Berapa nilai silhouette coefficient dan DBI dari cluster yang dihasilkan oleh DBSCAN? (10 poin)

## Feature Selection 2 (30 poin)
Setelah mengetahui performa hasil clustering pada model KMeans dan DBSCAN berdasakan silhouette coefficient dan DBI, lakukan proses feature selection untuk meningkatkan nilai silhouette coefficient dan DBI. Anda diperbolehkan untuk menggunakan metode iterative process atau metode yang lain.

1. Fitur apa yang memberikan nilai silhouette coefficient/DBI terbaik? Lengkapi dengan proses pemilihan fitur
2. Mengapa Anda menggunakan metode pemilihan fitur tersebut?


## BONUS - CHALLENGE (30 poin)
Buatlah model ANNOY menggunakan data yang telah melalui proses pra pengolahan data. Lakukan query dengan 5 nilai terdekat. Apakah ANNOY melakukan query pada cluster terbaik yang telah Anda bentuk?

# Jawab

## Pra Pengolahan Data (10 poin)

1. Load data (0 poin)
2. Cek deskripsi data apakah menggandung nilai yang hilang. Jika ya, sebutkan berapa dan pada fitur yang mana. (5 poin)
4. Drop fitur yang tidak relevan untuk proses clustering. (5 poin)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 1. Load data
data = pd.read_csv('/content/drive/MyDrive/Machine Learning/3_heart.csv')
data.info()

2. Cek deskripsi data apakah menggandung nilai yang hilang. Jika ya, sebutkan berapa dan pada fitur yang mana.
3. Drop fitur yang tidak relevan untuk proses clustering. (5 poin)

In [ ]:
# Cek nilai hilang

missing_values = data.isnull().sum()
print(missing_values)
print(f"Total missing: {missing_values.sum()}")

# tidak ada data yang kosong

X = data.drop(columns=['target'])

data.info()

## Feature Selection (20 poin)

1. Pilih 5 variabel terbaik menurut Anda dari seluruh variabel yang ada. (10 poin)
2. Bagaimana cara Anda memilih 5 variabel tersebut? Kemukakan alasan Anda. (10 poin)

1. Memilih 5 variabel terbaik

In [ ]:
# mencari 5 variabel terbaik menggunakan variance
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold()
selector.fit(X)
variances = selector.variances_
feature_variance = pd.DataFrame({
    'feature': X.columns,
    'variance': variances
}).sort_values('variance', ascending=False)

print("Ranking fitur berdasarkan variance:")
print(feature_variance)

# memilih 5 fitur dengan variasi tertinggi
top5_features = feature_variance.head(5)['feature'].tolist()
print(top5_features)

2. Cara pemilihan

Cara menentukan 5 variabel terbaik adalah menggunakan Variance Threshold, yaitu mengukur varian nilai yang tersebar dalam suatu fitur. Cara kerjanya yaitu tiap fitur dihitung variannya, kemudian diubah ke dalam array, dilakukan perankingan, dan diambil 5 fitur dengan varian terbesar.

## Clustering - 1 (30 poin)

1. Lakukan pembuatan model cluster dengan menggunakan KMeans dengan jumlah cluster mulai 1 hingga 10. (10 poin)
2. Buat diagram elbow, tentukan jumlah cluster terbaik menurut elbow method dan berikan alasan. (10 poin)
3. Hitung nilai silhouette coefficient dari k=1 hingga k=10. Berdassarkan nilai silhouette, berapa jumlah cluster terbaik? Apakah sama dengan metode elbow? (10 poin)

NB: Anda diperbolehkan untuk melakukan proses normalisasi/standardisasi sebelum pembuatan model clustering. Lakukan pengamatan terhadap hasil clustering.

In [ ]:
from sklearn.preprocessing import StandardScaler

# mengambil 5 fitur terbaik
final_features = top5_features
X = data[final_features]

print(X.head())

# normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_data = pd.DataFrame(X_scaled, columns=final_features)
print("Data setelah standardisasi:")
print(X_scaled_data.head())

1. Lakukan pembuatan model cluster dengan menggunakan KMeans dengan jumlah cluster mulai 1 hingga 10. (10 poin)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# membuat model cluster dengan jumlah 1-10
sse = []
silhouette_scores = [] # Renamed the list to avoid conflict
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    labels = kmeans.labels_
    sse.append(kmeans.inertia_)

2. Buat diagram elbow, tentukan jumlah cluster terbaik menurut elbow method dan berikan alasan. (10 poin)

In [ ]:
# Visualisasi diagram elbow
plt.figure(figsize=(8,4))
plt.plot(k_range, sse, "bx-")
plt.xlabel("k")
plt.ylabel("sse")
plt.title("Elbow Method to Find The Best K")
plt.grid(True)
plt.show()

# Analisis penurunan nilai SSE
for i in range(1, len(sse)):
    reduction = sse[i-1] - sse[i]
    reduction_pct = (reduction / sse[i-1]) * 100

    # Tandai di mana penurunan mulai tidak signifikan
    if reduction_pct < 15:
        indicator = "← PENURUNAN TIDAK SIGNIFIKAN"
    elif reduction_pct < 25:
        indicator = "← PENURUNAN MULAI MELAMBAT"
    else:
        indicator = ""

    print(f"{i}→{i+1}\t{sse[i]:.2f}\t\t{reduction:.2f}\t\t{reduction_pct:.1f}%\t{indicator}")



Dari model KMeans yang kemudian menggunakan metode elbow, nilai k optimal untuk dataset ini adalah 2. Hal ini dapat dibuktikan dari penurunan nilai SSE tidak signifikan dimulai dari titik ke-2.

3. Hitung nilai silhouette coefficient dari k=1 hingga k=10. Berdassarkan nilai silhouette, berapa jumlah cluster terbaik? Apakah sama dengan metode elbow? (10 poin)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# membuat model cluster dengan jumlah 1-10
sse = []
silhouette_scores = [] # Renamed the list to avoid conflict
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    labels = kmeans.labels_
    sse.append(kmeans.inertia_)

    # mencari nilai silhouette
    if k > 1:
        sil_score = silhouette_score(X_scaled, labels)
        silhouette_scores.append(sil_score)
        print(f"k={k}: Silhouette Score = {sil_score:.4f}")
    else:
        silhouette_scores.append(0)  # Tidak terdefinisi untuk k=1
        print(f"k={k}: Silhouette Score = Tidak terdefinisi")

Jika menggunakan nilai Silhouette Score, nilai k terbaik adalah ketika k=2.

Setelah melakukan perhitungan menggunakan metode elbow dan nilai Silhouette, terdapat kesamaan nilai k optimal diantara kedua metode ini yaitu k=2.

**Model Final KMeans**

In [ ]:
optimal_k = 2

# Model final KMeans
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
final_labels = kmeans.fit_predict(X_scaled)

## Clustering - 2 (20 poin)

1. Lakukan pembuatan model cluster dengan menggunakan DBSCAN. Berapa jumlah cluster yang dihasilkan? (10 poin)
2. Berapa nilai silhouette coefficient dan DBI dari cluster yang dihasilkan oleh DBSCAN? (10 poin)

In [ ]:
# pembuatan model cluster dengan DBSCAN
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score

# membuat model DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
labels = dbscan.fit_predict(X_scaled)

# menghitung jumlah cluster
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)

print(f"Jumlah cluster: {n_clusters}")
print(f"Jumlah outlier: {n_noise}")

# menghitung metrik dengan menggunakan data non-noise
non_noise_mask = labels != -1
if len(set(labels[non_noise_mask])) > 1:
  X_non_noise = X_scaled[non_noise_mask]
  labels_non_noise = labels[non_noise_mask]

  sil_score = silhouette_score(X_non_noise, labels_non_noise)
  dbi_score = davies_bouldin_score(X_non_noise, labels_non_noise)

  print(f"Silhouette Score: {sil_score}")
  print(f"DBI Score: {dbi_score}")

**Model DBSCAN**

In [ ]:
print("=== MODEL DBSCAN SUDAH TERBENTUK ===")
print(f"✓ Parameter: eps={dbscan.eps}, min_samples={dbscan.min_samples}")
print(f"✓ Jumlah cluster: {n_clusters}")
print(f"✓ Jumlah noise: {n_noise}")
print(f"✓ Core samples: {len(dbscan.core_sample_indices_)}")
print(f"✓ Labels: {len(labels)} data points")

# Distribusi cluster detail
print(f"\n=== DISTRIBUSI DETAIL ===")
unique, counts = np.unique(labels, return_counts=True)
for label, count in zip(unique, counts):
    if label == -1:
        print(f"  Noise: {count} points")
    else:
        print(f"  Cluster {label}: {count} points")

## BONUS - CHALLENGE (30 poin)
Buatlah model ANNOY menggunakan data yang telah melalui proses pra pengolahan data. Lakukan query dengan 5 nilai terdekat. Apakah ANNOY melakukan query pada cluster terbaik yang telah Anda bentuk?

In [ ]:
%pip install annoy

In [ ]:
from annoy import AnnoyIndex

# persiapan data menggunakan 5 fitur terbaik
n_features = X_scaled.shape[1] # mengambil 5 fitur terbaik
n_trees = 50

# membuat ANNOY index
annoy_index = AnnoyIndex(n_features, 'euclidean')

# menambah semua data points ke index
for i, vector in enumerate(X_scaled):
    annoy_index.add_item(i, vector)

# membuat index dengan n trees
annoy_index.build(n_trees)
print(f"ANNOY index built with {n_trees} trees")

# Test query
print("\n=== TEST QUERY ===")
query_idx = 0  # Data point pertama
actual_cluster = final_labels[query_idx]

print(f"Query point {query_idx} - Actual cluster: {actual_cluster}")

# Get 5 nearest neighbors
neighbors, distances = annoy_index.get_nns_by_item(query_idx, 6, include_distances=True)
neighbors = neighbors[1:]  # Exclude self
distances = distances[1:]

print("5 Nearest Neighbors:")
for i, (neighbor, dist) in enumerate(zip(neighbors, distances)):
    neighbor_cluster = final_labels[neighbor]
    match = "✓ SAME" if neighbor_cluster == actual_cluster else "✗ DIFFERENT"
    print(f"  {i+1}. Point {neighbor} (Cluster {neighbor_cluster}) - Dist: {dist:.3f} {match}")

# Hitung akurasi
matches = sum(1 for n in neighbors if final_labels[n] == actual_cluster)
accuracy = matches / len(neighbors) * 100
print(f"Cluster matching accuracy: {accuracy:.1f}%")


Ya, ANNOY melakukan query pada cluster terbaik yang telah dibuat.